In [17]:
#segment tree(prototype)
#range minimum query ver
#recursive
#デフォルトの値を適宜変えて
###0based index
class segment_tree():
    def __init__(self,size):
        self.size = 2**(size-1).bit_length() #完全二分木にするためsizeを2冪にする
        self.iden = float("inf") #identity_element これで初期化する
        self.dat = [self.iden]*(self.size*2-1) #(2n-1)のleaf
        
    def update(self,ind,x): #更新する場所,更新する値
        #子から順に上っていく。
        ind += self.size - 1 #ind番目の要素は(0から始まる)
        self.dat[ind] = x
        while ind > 0:
            ind = (ind-1) >> 1 #(//2) #親のindex
            self.dat[ind] = min(self.dat[ind*2+1],self.dat[ind*2+2])
    
    def query(self,q_le,q_ri,cur=0,LEFT=0,RIGHT=None):  #[q_le,q_ri)　区間内での目的の値を取得
        #親から順に(root)下っていく
        if RIGHT is None:          #initializing
            RIGHT = self.size
        if RIGHT <= q_le or q_ri <= LEFT: #範囲外の場合
            return self.iden
        elif q_le <= LEFT and RIGHT <= q_ri: #範囲内の場合
            return self.dat[cur]
        else:   #一部区間の場合
            lres = self.query(q_le,q_ri,cur*2+1, LEFT, (LEFT+RIGHT)//2)
            rres = self.query(q_le,q_ri,cur*2+2, (LEFT+RIGHT)//2,RIGHT)
            return min(lres,rres)
a = segment_tree(8)

a.update(7,3)
print(a.dat)
a.query(7,8)

[3, inf, 3, inf, inf, inf, 3, inf, inf, inf, inf, inf, inf, inf, 3]


3

In [2]:
#standard segment tree
#1based index
#not recursive
#[l,r)

def segfunc(x,y):   #operator
    return min(x,y) #change this function such as x+y, max(x,y)

class SegTree:
    def __init__(self,ini_lis,segfunc,ide):  #initial_list, operator, identity_elements
        n = len(ini_lis)                  #要素の数
        self.segfunc = segfunc            #演算子
        self.ide = ide                    #単位元
        self.num = 2**((n-1).bit_length)  #leafの数(完全二分木にする)
        self.tree = [ide] * 2 * self.num  #1-indexにするため *2([0]を無視)

        for i in range(n):
            self.tree[i] = ini_lis[i]
        for i in range(self.num - 1, 0, -1): #leafじゃないnode数はleaf-1
            self.tree[i] = self.segfunc(self.tree[2*i], self.tree[2*i + 1])

    def add(self,k,x):  #k番目の要素ににxをaddition
        k += self.num   #k番目の要素はtree上ではk+num
        self.tree[k] += x
        while k > 1:
            self.tree[k>>1] = self.segfunc(self.tree[k],self.tree[k^1])
            k >>= 1
    
    def update(self,k,x): #k番目の要素をxにrepalce
        k += self.num
        self.tree[k] = x
        while k > 1:
            self.tree[k>>1] = self.segfunc(self.tree[k],self.tree[k^1])
            k >>= 1
    
    def query(self,l,r): #[l,r)区間のquery bottom up, not recursive
        res = self.ide
        l += self.num
        r += self.num
        while l < r:
            if l & 1:   #二分に分かれたのnodeのうち右側の時，そのノードの値を拾う必要がある．
                res = self.segfunc(res,self.tree[l])
                l += 1
            if r & 1:   #区間は,r)なので，実質的にrが二分に分かれたnodeのうち左側の時，同様
                res = self.segfunc(res,self.tree[r-1])
            l >>= 1     #親へ移動
            r >>= 1
        return res

        a = SegTree([1,2,3,4],segfunc,0)
        a.

2